In [ ]:
!pip install tensorflow_federated==0.13.1

     |████████████████████████████████| 428 kB 14.3 MB/s 
     |████████████████████████████████| 104 kB 83.3 MB/s 
     |████████████████████████████████| 93 kB 1.6 MB/s 
     |████████████████████████████████| 20.0 MB 921 kB/s 
     |████████████████████████████████| 1.0 MB 51.3 MB/s 
     |████████████████████████████████| 2.2 MB 61.8 MB/s 
     |████████████████████████████████| 82 kB 763 kB/s 
     |████████████████████████████████| 2.8 MB 78.7 MB/s 
     |████████████████████████████████| 422.0 MB 34 kB/s 
     |████████████████████████████████| 41 kB 562 kB/s 
     |████████████████████████████████| 50 kB 5.7 MB/s 
     |████████████████████████████████| 448 kB 33.1 MB/s 
     |████████████████████████████████| 3.8 MB 88.8 MB/s 
  Created wheel for absl-py: filename=absl_py-0.9.0-py3-none-any.whl size=121940 sha256=2722d06f0b9536658a0ebeef99315ff4b0baad2b01870d915732b41915e8b745
  Stored in directory: /root/.cache/pip/wheels/cc/af/1a/498a24d0730ef484019e007bb9e8cef3ac00311a672c0

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

n_instances = 4 # the number of instance each party has
n_parties = 100 # number of parties

# load data and transform it
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

x_train = np.concatenate( [ x_train[ np.ravel(y_train == 0) ][ :200 ], x_train[ np.ravel(y_train == 1) ][ :200 ] ] ).reshape( (400,-1) )
x_test =  np.concatenate( [ x_test[ np.ravel(y_test == 0) ][ :100 ], x_test[ np.ravel(y_test == 1) ][ :100 ] ] ).reshape( (200,-1) )
x_train = x_train.astype( float ) / 255.
x_test = x_test.astype( float ) / 255.

print( 'training data: ', x_train.shape )
print( 'test data: ', x_test.shape )

# labels
y_train = np.concatenate( [ np.zeros( 200 ), np.ones( 200 ) ] )
y_test = np.concatenate( [ np.zeros( 100 ), np.ones( 100 ) ] )

idx = np.arange( len( x_train ) )
np.random.shuffle( idx )
x_train = x_train[ idx ]
y_train = y_train[ idx ]

# gather the data for each party
data_x = [ x_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_parties ) ]
data_y = [ y_train[ i * n_instances : ( i + 1 ) * n_instances ] for i in range( n_parties ) ]

training data:  (400, 3072)
test data:  (200, 3072)


In [ ]:
# initialize the weights of our perceptron
weights = np.random.random( size=3072 ) * 2 - 1
 
# learning rate
lr = 0.1
# batch size
bs = 2
pred=[]
# the training function
def train( weights, x, y ):
  # train for a number of epochs
  for e in range( 10 ):
    acc = [] 
    print( 'epoch', e )
    # the actual sgd
    for i in range( len( x ) // bs ):
      batch_x = x[ i * bs : (i+1) * bs ]
      batch_y = y[ i * bs : (i+1) * bs ]
      
      # forward step
      f = np.sum( batch_x * weights, axis=1 )
      
      # apply sigmoid
      f = 1 / ( 1 + np.exp( -f ) )  
      
      # calculate the error
      error = f - batch_y

      # calculate gradient
      g = batch_x.T.dot( error ) / batch_x.shape[ 0 ]
      
      # weight update 
      weights -= lr * g

      # calculate acc
      preds = (f > 0.5).astype( int )  
      batch_acc = np.sum( preds == batch_y ) / bs
      acc.append( batch_acc )
    print( 'acc: ', sum( acc ) / len( acc ) )
    
  # test data
  prediction = ( np.sum( x_test * weights, axis=1 ) > 0.5 ).astype( int )
  print( 'test acc:', np.sum( prediction == y_test ) / len( y_test )  )

train( weights, data_x[ 0 ], data_y[ 1 ] )

epoch 0
acc:  0.5
epoch 1
acc:  1.0
epoch 2
acc:  1.0
epoch 3
acc:  1.0
epoch 4
acc:  1.0
epoch 5
acc:  1.0
epoch 6
acc:  1.0
epoch 7
acc:  1.0
epoch 8
acc:  1.0
epoch 9
acc:  1.0
test acc: 0.42


In [ ]:
# the local weights of every party 
models = [ np.random.random( size=3072 ) * 2 - 1 for _ in range( n_parties ) ]

def get_gradient( weights, x, y ):
  """
  returns the gradients wrt. loss for the training samples
  """
  grads = []
  for i in range( len( x ) // bs ):
    batch_x = x_train[ i * bs : (i+1) * bs ]
    batch_y = y_train[ i * bs : (i+1) * bs ]
    # forward step
    f = np.sum( batch_x * weights, axis=1 )
    # sigmoid
    f = 1 / ( 1 + np.exp( -f ) )  
    # loss
    error = f - batch_y
    # gradient
    g = batch_x.T.dot( error ) / batch_x.shape[ 0 ]
    # save the gradients from this batch 
    grads.append( g )  
  
  # return the average of the gradients
  return sum( grads ) / len( grads )

# the servers weights
w = np.random.random( size=3072 ) * 2 - 1
# do 10 epochs
for i in range( 20 ):
  # get all gradients from the parties
  gradients = [ get_gradient( models[ i ], data_x[ i ], data_y[ i ] ) for i in range( n_parties ) ]

  # average gradients
  gradients = sum( gradients ) / len( gradients )
  # do weight updates
  w -= lr * gradients
  # test data
  prediction = ( np.sum( x_test * w, axis=1 ) > 0.5 ).astype( int )
  print( 'test acc:', np.sum( prediction == y_test ) / len( y_test )  )


test acc: 0.63
test acc: 0.58
test acc: 0.545
test acc: 0.525
test acc: 0.52
test acc: 0.515
test acc: 0.51
test acc: 0.505
test acc: 0.505
test acc: 0.505
test acc: 0.505
test acc: 0.505
test acc: 0.505
test acc: 0.505
test acc: 0.505
test acc: 0.505
test acc: 0.505
test acc: 0.505
test acc: 0.505
test acc: 0.505


In [ ]:
# the local weights of every party 
models = [ np.random.random( size=3072 ) * 2 - 1 for _ in range( n_parties ) ]

# shared masks. 0 and 5, 1 and 6, etc share a mask
masks = [ np.random.random( size=3072 ) * 2 - 1 for _ in range( n_parties // 2 ) ]
masks += masks


# learning rate
lr = 0.09
# batch size
bs = 2

def get_gradient_masked( weights, x, y, id ):
  grads = []
  for i in range( len( x ) // bs ):
    batch_x = x_train[ i * bs : (i+1) * bs ]
    batch_y = y_train[ i * bs : (i+1) * bs ]
    # forward step
    f = np.sum( batch_x * weights, axis=1 )
    # sigmoid
    f = 1 / ( 1 + np.exp( -f ) )  
    # loss
    error = f - batch_y
    # gradient
    g = batch_x.T.dot( error ) / batch_x.shape[ 0 ]
    grads.append( g )  

  g = sum( grads ) / len( grads )
  # add the mask
  if id < 5:
    g += masks[ id ]
  else:
    g -= masks[ id ]
  return g

w = np.random.random( size=3072 ) * 2 - 1
for i in range( 10 ):
  # get all gradients
  gradients = [ get_gradient_masked( models[ i ], data_x[ i ], data_y[ i ], i ) for i in range( n_parties ) ]

  # average gradients
  gradients = sum( gradients ) / len( gradients )
  # do weight updates
  w -= lr * gradients
  # test data
  prediction = ( np.sum( x_test * w, axis=1 ) > 0.5 ).astype( int )
  print( 'test acc:', np.sum( prediction == y_test ) / len( y_test )  )



test acc: 0.37
test acc: 0.425
test acc: 0.445
test acc: 0.475
test acc: 0.505
test acc: 0.515
test acc: 0.515
test acc: 0.52
test acc: 0.505
test acc: 0.505


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()
x_train = np.concatenate( [ x_train[ np.ravel(y_train == 0) ], x_train[ np.ravel(y_train == 1 )] ] )
x_test = np.concatenate( [ x_test[ np.ravel(y_test == 0) ], x_test[ np.ravel(y_test == 1) ] ] )
x_train = x_train.astype( float ) / 255.
x_test = x_test.astype( float ) / 255.

x_train = x_train.reshape( ( x_train.shape[ 0 ], -1  ) )
x_test = x_test.reshape( ( x_test.shape[ 0 ], -1  ) )
print( x_train.shape )

# labels
y_train = np.concatenate( [ np.zeros( np.sum( y_train == 0 ) ), np.ones( np.sum( y_train == 1  ) ) ]  )
y_test = np.concatenate( [ np.zeros( np.sum( y_test == 0 ) ), np.ones( np.sum( y_test == 1 ) ) ]  ) 

(1000, 3072)


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

tf.get_logger().setLevel('INFO')

# parameters
NO_CLIENTS = 3 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NO_CLIENT_SAMPLES = TOTAL_SAMPLES // NO_CLIENTS # number of samples per client
BATCH_SIZE = 64
EPOCHS = 10

# shuffle data
idx = np.arange( TOTAL_SAMPLES )
np.random.shuffle( idx )
x_train = x_train[ idx ]
y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NO_CLIENTS ):
  x = x_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )


# define a function that builds our model
def build_model():
  model = tf.keras.models.Sequential()

  model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
  model.add( tf.keras.layers.Dense( 2, activation='softmax' ) )

  return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )


# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
for e in range( EPOCHS ):
  state, metrics = algorithm.next( state, data )
  print( 'epoch' , e , metrics )

(333, 3072)
(333, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(333, 3072)
(333, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(333, 3072)
(333, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

epoch 0 <sparse_categorical_accuracy=0.7680680751800537,loss=0.489005446434021,keras_training_time_client_sum_sec=0.0>